In [4]:
#pip install lxml

     |████████████████████████████████| 5.5MB 6.5MB/s eta 0:00:01     |██████▌                         | 1.1MB 6.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


#### This will grab data table from wikipedia website and place it in a pandas dataframe

In [6]:
import pandas as pd
myData1 =  pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = myData1[0]
df.rename(columns = {'District' : 'Borough'}, inplace = True)
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [2]:
df.shape

(180, 3)

#### This code will filter out all the boroughs labeled Not assigned and save to a new dataframe
#### #Probably not best idea for huge datasets...

In [7]:
my_df = df[df['Borough'] != 'Not assigned']
my_df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
my_df.shape

(103, 3)

In [27]:
ll_df = pd.read_csv('https://cocl.us/Geospatial_data')
ll_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### This code merges the two datasets into a new one that will be used for the final bit
### Found the code syntax example on stackoverflow.com

In [29]:
new_df = my_df.join(ll_df.set_index(['Postal Code'], verify_integrity = True), on = ['Postal Code'], how = 'left')
new_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [41]:
import folium
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [33]:
latitude = 43.65460
longitude = -79.360636
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_Toronto

In [34]:
CLIENT_ID = 'V4T0HDCPWZVSPMMHVTNJAT3QORC2H10MKVJEM44ZRDTY2XKW' # your Foursquare ID
CLIENT_SECRET = 'KXYRVUGUUAMPJTPGNSQD31202ZB0ZPSQGHUKZVKWGHRK5KB2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Define Downtown Toronto

In [37]:
#Define Downtown Toronto
DT_latitude = 43.65
DT_longitude = -79.38

### Brief analysis of Downton Toronto Neighborhood using FourSquare Data stuff

In [39]:
#FourSquare DATA for connecting to downtown_Toronto
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    DT_latitude, 
    DT_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=V4T0HDCPWZVSPMMHVTNJAT3QORC2H10MKVJEM44ZRDTY2XKW&client_secret=KXYRVUGUUAMPJTPGNSQD31202ZB0ZPSQGHUKZVKWGHRK5KB2&v=20180605&ll=43.65,-79.38&radius=500&limit=100'

In [42]:
#Get information from FourSquares
results = requests.get(url).json()


### Borrowed this function from NY lab to clean data coming from FourSquare

In [44]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [45]:
venues = results['response']['groups'][0]['items']   
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Richmond Station,American Restaurant,43.651569,-79.379266
1,Adelaide Club Toronto,Gym / Fitness Center,43.649279,-79.381921
2,Dineen Coffee,Café,43.650497,-79.378765
3,Equinox Bay Street,Gym,43.648100,-79.379989
4,GoodLife Fitness Toronto 137 Yonge Street,Gym,43.651242,-79.378068


In [46]:
nearby_venues.shape

(100, 4)

### Beginning Analysis of some sort for the FourSquare Data
### This is not very clear :-\

In [49]:
# one hot encoding
Toronto_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Name'] = nearby_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Name,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Beer Bar,Bookstore,Breakfast Spot,Burger Joint,...,Smoothie Shop,Steakhouse,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Richmond Station,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adelaide Club Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dineen Coffee,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Equinox Bay Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,GoodLife Fitness Toronto 137 Yonge Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
Toronto_onehot.shape

(100, 58)